In [1]:
import numpy as np
import pandas as pd
import math
import glob
import time
import csv

# All files and directories ending with .txt and that don't begin with a dot:


In [2]:
def executaTodosTestes():
    testFiles = glob.glob("Testes/*")
    with open('resultados.csv','w', newline='') as f:
        escrever = csv.writer(f,delimiter=';')
        escrever.writerow(['#Nome: Leandro Diniz Silva'])
        escrever.writerow(['#Matricula: 2019105718'])
        escrever.writerow(['Teste','Resultado BackTrack', 'Tempo BackTrack',
                          'Resultado BranchBound', 'Tempo BranchBound'])
        for file in testFiles:
            df = lerDatParaPanda(file)
            n = int(df[0][0])
            capacidade = int(df[0][1])
            df = np.delete(df, (0), axis=0)      

            start = time.time()
            bt = mochilaBackTrack(df, capacidade, n)
            end = time.time()
            tempobt = end - start

            start = time.time()
            bb = mochilaBranchBoundMain(df, capacidade, n)
            end = time.time()
            tempobb = end - start
            escrever.writerow([file[7:],bt, tempobt,bb, tempobb])

            print("Teste: ",file[7:])
            print("BackTrack:", bt)
            print("Tempo BackTrack:", tempobt)
            print("Branch and Bound", bb)
            print("Tempo Branch and Bound:", tempobb, "\n")

In [3]:
def lerDatParaPanda(caminhoArquivo):
    df = pd.read_csv(caminhoArquivo, 
                 sep=" ", comment='@', header=None)
    df[0] = pd.to_numeric(df[0], downcast="float")
    df[1] = pd.to_numeric(df[1], downcast="float")
    df = df.round(2)
    return df.values

In [4]:
class Node:
    def __init__(self, level, valor, peso, limite):
         self.level = level
         self.valor = valor
         self.peso = peso
         self.limite = limite

In [5]:
def mochilaBackTrack(data, cap, contador):
    if (cap == 0 or (contador - 1) < 0):
        return 0

    #Se o peso do valor atual for maior que a capacidade,
    # "pula" ele
    if (data[contador - 1][1] > cap):
        return mochilaBackTrack(data, cap, contador -1)
    
    return max(data[contador -1][0] + mochilaBackTrack(data, (cap - data[contador-1][1]), contador -1),
                   mochilaBackTrack(data, data[contador-1][1], contador -1))

In [6]:
def nodeBound(node, data, cap, contador):
    #Se o peso do node atual for maior que a capacidade
    #"pula" ele
    if (node.peso > cap) :
        return 0
    
    valorLimite = node.valor
    
    auxA = node.level + 1
    pesoTotal = node.peso
    
    while ((auxA < contador) and (pesoTotal + data[auxA][1] <= cap)):
        pesoTotal += data[auxA][1]
        valorLimite += data[auxA][0]
        auxA = auxA + 1
    
    if (auxA < contador):
        valorLimite += (cap - pesoTotal) * data[auxA][2];
  
    return valorLimite;

In [7]:
def mochilaBranchBoundMain(data, cap, contador):
    #Sort descendente no array para o com o maior valor ficar no topo
    values = data[:,1] / data[:,0]
    data = np.column_stack((data, values))
    data = data[data[:,-1].argsort()]
    data = data[::-1]
    return mochilaBranchBound(data,cap,contador)

In [8]:
def mochilaBranchBound(data, cap, contador):
    queue = []
    u = Node(0,0,0,0)
    v = Node(0,0,0,0)
    u.level = -1
    u.valor = u.peso = 0
    queue.append(u)

    melhorCapacidade = 0
    #while queue != 0 do
    while (queue):
        #node = queue.pop()
        u = queue.pop()
        
        if (u.level == -1):
            v.level = 0
        #if node.level == n - 1
        if (u.level == contador-1):
            continue
        #end

        #with = node.value + ...
        v.level = u.level + 1
        v.peso = u.peso + data[v.level][1]
        v.valor = u.valor + data[v.level][0]
        #esse v é o without, pois ele pula 1 level no "v.level = u.level +1"
        
        #if (node.weight + items ... AND with > ...)
        if ((v.peso <= cap) and (v.valor > melhorCapacidade)):
            melhorCapacidade = v.valor

        v.limite = nodeBound(v, data, cap, contador)
  
        if (v.limite > melhorCapacidade):
            queue.append(v)

        v.peso = u.peso
        v.valor = u.valor
        v.limite = nodeBound(v, data, cap, contador)
        #esse v é o with, pois ele volta ao seu estado original "u"
        if (v.limite > melhorCapacidade):
            queue.append(v)

    return melhorCapacidade

In [9]:
executaTodosTestes()

Teste:  f10_l-d_kp_20_879
BackTrack: 798.0
Tempo BackTrack: 0.004012107849121094
Branch and Bound 679.0
Tempo Branch and Bound: 0.0 

Teste:  f1_l-d_kp_10_269
BackTrack: 293.0
Tempo BackTrack: 0.0
Branch and Bound 114.0
Tempo Branch and Bound: 0.0 

Teste:  f2_l-d_kp_20_878
BackTrack: 878.0
Tempo BackTrack: 0.003011465072631836
Branch and Bound 678.0
Tempo Branch and Bound: 0.0010023117065429688 

Teste:  f3_l-d_kp_4_20
BackTrack: 28.0
Tempo BackTrack: 0.0
Branch and Bound 35.0
Tempo Branch and Bound: 0.0 

Teste:  f4_l-d_kp_4_11
BackTrack: 23.0
Tempo BackTrack: 0.0
Branch and Bound 22.0
Tempo Branch and Bound: 0.0 

Teste:  f5_l-d_kp_15_375
BackTrack: 408.81999933719635
Tempo BackTrack: 0.0010025501251220703
Branch and Bound 96.529998421669
Tempo Branch and Bound: 0.0 

Teste:  f6_l-d_kp_10_60
BackTrack: 50.0
Tempo BackTrack: 0.0
Branch and Bound 33.0
Tempo Branch and Bound: 0.0 

Teste:  f7_l-d_kp_7_50
BackTrack: 79.0
Tempo BackTrack: 0.0
Branch and Bound 88.0
Tempo Branch and Bound: